In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

import joblib
import time
start_time = time.time()
import warnings
warnings.filterwarnings('ignore')

# Load dataset
data_file = "merged.csv"
df = pd.read_csv(data_file)

cols = [
    "Bwd Packet Length Std", "Flow Bytes/s", "Total Length of Fwd Packets", "Fwd Packet Length Std",
    "Flow IAT Std", "Flow IAT Min", "Fwd IAT Total", "Flow Duration", "Bwd Packet Length Max",
    "Flow IAT Max", "Flow IAT Mean", "Total Length of Bwd Packets", "Fwd Packet Length Min",
    "Bwd Packet Length Mean", "Flow Packets/s", "Fwd Packet Length Mean", "Total Backward Packets",
    "Fwd Packet Length Max", "Total Fwd Packets", "Bwd Packet Length Min", 'Label'
]

df = df[cols].fillna(0)
df['Label'] = df['Label'].apply(lambda x: 1 if x == "BENIGN" else 0)

X_full = df.drop("Label", axis=1)
y_full = df["Label"]

# Standardize and apply PCA on 4 components
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_full)

pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_scaled)

# Train-test split (using 500 samples)
X_train, _, y_train, _ = train_test_split(X_pca, y_full, train_size=1000, stratify=y_full, random_state=42)

# Quantum kernel setup using FidelityQuantumKernel
feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement='linear')
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
quantum_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

qsvc = QSVC(quantum_kernel=quantum_kernel)

# Train model
qsvc.fit(X_train, y_train)
y_pred = qsvc.predict(X_train)

print("\nClassification Report:")
print(classification_report(y_train, y_pred))

# Save model, scaler, and PCA
joblib.dump(qsvc, "Result/network_anomaly_QSVC.pkl")
joblib.dump(scaler, "Result/qsvc_scaler.pkl")
joblib.dump(pca, "Result/qsvc_pca_transform.pkl")
joblib.dump(cols[:-1], "Result/qsvc_features_used.pkl")

print("\nModel and transformers saved successfully.")
print("Time taken:", round(time.time() - start_time, 2), "seconds")


Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.16      0.27       167
           1       0.86      0.99      0.92       833

    accuracy                           0.85      1000
   macro avg       0.84      0.58      0.59      1000
weighted avg       0.85      0.85      0.81      1000


Model and transformers saved successfully.
Time taken: 73278.62 seconds
